Importing Libraries


In [1]:
# Importing necessary libraries
from os import listdir
from matplotlib import image
import os
import numpy as np
from matplotlib import image
import cv2 as cv
from skimage.feature import greycomatrix, greycoprops
import matplotlib.pyplot as plt
import itertools
import scipy.spatial.distance as dist
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn import metrics


Loading Images

In [ ]:
# Load all images in a directory
loaded_images = list()
loaded_images2 = list()

path = './angry'
path1 = './disgusted'

# Load images for 'angry' class
for filename in listdir(path):
    img_data = image.imread(path + '/' + filename)
    loaded_images.append(img_data)

# Load images for 'disgusted' class
for filename in listdir(path1):
    img_data = image.imread(path1 + '/' + filename)
    loaded_images2.append(img_data)


Defining Texture Features Function

In [ ]:
def textureFeatures(img):
    """Based on texture analysis using GLCM (Gray-Level Co-Occurrence Matrix).
    The feature vector of size 1x4 will contain [Contrast, Correlation, Energy, Homogeneity]
    """
    features_texture = []
    im = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

    # Normalize before returning the descriptor vector
    glcm = greycomatrix(im, distances=[5], angles=[0], symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')
    corr = greycoprops(glcm, 'correlation')
    ener = greycoprops(glcm, 'energy')
    homog = greycoprops(glcm, 'homogeneity')

    features1 = np.append(contrast, corr)
    features2 = np.append(ener, homog)
    features_texture = np.append(features1, features2)
    features_texture = features_texture / np.mean(features_texture)

    return features_texture


Defining Feature Extraction Function

In [ ]:
def getFeatures(img, fsize):
    """Function to create a descriptor vector.
    """
    features = []
    if fsize >= 43:
        # Using texture features
        features = np.append(textureFeatures(img))
    return features


Defining CBIR Indexation Function

In [ ]:
def CBIR_Indexation(fsize):
    """Content-Based Image Retrieval (CBIR) Indexation."""
    n = len(loaded_images)
    m = len(loaded_images2)
    features = []
    data = np.arange(n + m)
    y = []
    print("Debut d'Indexation")

    for i in range(n):
        features.append(getFeatures(loaded_images[i], fsize))
        y.append('angry')

    for j in range(m):
        features.append(getFeatures(loaded_images2[j], fsize))
        y.append('disgusted')

    print("Fin d'Indexation")
    return features, y


 Train-Test Split and SVM Classification

In [ ]:
# Indexing and creating feature and label matrices
index_Matrix = CBIR_Indexation(43)
X = index_Matrix[0]
Y = index_Matrix[1]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=35)

# Initializing and training the SVM classifier
clf = svm.SVC(C=9, kernel='linear')
clf.fit(X_train, y_train)

# Predictions and accuracy
print('prediction :', clf.predict(X_test))
print("Accuracy  SVM: %s" % clf.score(X_test, y_test))

# Confusion matrix
results = confusion_matrix(y_test, clf.predict(X_test))
print('Matrix confusion \n', results)
